# AgentCore Travel Assistant - Memory Demo

Interactive demonstration of cross-session memory with Amazon Bedrock AgentCore Runtime. This notebook shows how the deployed agent maintains context across sessions using persistent memory.

## What You'll Learn

- Invoke a deployed AgentCore agent programmatically
- Test short-term memory within a session
- Test long-term memory across different sessions
- Send multimodal content (images and videos)
- Understand session and user ID management with custom headers

## Prerequisites

- Deployed AgentCore agent (see `../deployment/`)
- Agent ARN from deployment
- AWS credentials configured
- Python 3.10+ with boto3 installed

## Setup

### Install Dependencies

In [ ]:
!pip install boto3 -q

### Import Libraries

In [ ]:
import boto3
import json
import uuid
import base64
import os
from datetime import datetime

print("✅ All imports successful!")

## Configuration

**Important**: Replace `YOUR_AGENT_ARN` with your actual agent ARN from deployment.

In [ ]:
# AWS Configuration
AWS_REGION = 'us-east-1'  # Change to your region
AGENT_ARN = "YOUR_AGENT_ARN"  # ⚠️ CHANGE THIS! Get from: agentcore status

# User and Session IDs (persistent across notebook restarts)
USER_ID = "demo-user-alice-12345678901234567890"  # Must be 33+ characters
SESSION_1_ID = "session-1-alice-travel-planning-12345678"  # Must be 33+ characters
SESSION_2_ID = "session-2-alice-follow-up-87654321"  # Must be 33+ characters

print(f"🌍 Region: {AWS_REGION}")
print(f"🤖 Agent ARN: {AGENT_ARN}")
print(f"👤 User ID: {USER_ID}")
print(f"📝 Session 1 ID: {SESSION_1_ID}")
print(f"📝 Session 2 ID: {SESSION_2_ID}")
print(f"\n⚠️  Note: Session and User IDs are persistent - memory will be maintained even if you restart this notebook!")

## Helper Functions

In [ ]:
# Initialize AgentCore client
agentcore_client = boto3.client('bedrock-agentcore', region_name=AWS_REGION)

def invoke_agent(prompt, session_id, user_id, media=None):
    """
    Invoke the AgentCore agent with text and optional media using custom headers.
    
    Args:
        prompt: Text message to send
        session_id: Session ID (33+ characters)
        user_id: User ID (33+ characters)
        media: Optional dict with 'type', 'format', 'data' (base64)
    
    Returns:
        Agent response as string
    """
    # Setup event handler for custom headers
    event_system = agentcore_client.meta.events
    EVENT_NAME = 'before-sign.bedrock-agentcore.InvokeAgentRuntime'
    CUSTOM_HEADER_NAME = 'X-Amzn-Bedrock-AgentCore-Runtime-Custom-Actor-Id'
    
    def add_custom_runtime_header(request, **kwargs):
        """Add custom header for user identification."""
        request.headers.add_header(CUSTOM_HEADER_NAME, user_id)
    
    # Prepare payload
    payload_data = {"prompt": prompt}
    if media:
        payload_data["media"] = media
    
    payload = json.dumps(payload_data).encode()
    
    try:
        # Register event handler
        handler = event_system.register_first(EVENT_NAME, add_custom_runtime_header)
        
        # Invoke agent
        response = agentcore_client.invoke_agent_runtime(
            agentRuntimeArn=AGENT_ARN,
            runtimeSessionId=session_id,
            payload=payload,
            qualifier="DEFAULT"
        )
        
        # Parse response
        content = []
        for chunk in response.get("response", []):
            content.append(chunk.decode('utf-8'))
        
        if content:
            response_text = ''.join(content)
            try:
                response_json = json.loads(response_text)
                return response_json.get("result", response_text)
            except json.JSONDecodeError:
                return response_text
        
        return "No response from agent"
    
    finally:
        # Unregister event handler
        event_system.unregister(EVENT_NAME, handler)

def print_conversation(role, message, session_id=None):
    """Pretty print conversation"""
    timestamp = datetime.now().strftime("%H:%M:%S")
    session_info = f" [Session: {session_id[:20]}...]" if session_id else ""
    print(f"\n{'='*80}")
    print(f"[{timestamp}] {role}{session_info}")
    print(f"{'='*80}")
    print(message)
    print(f"{'='*80}\n")

print("✅ Helper functions loaded with custom header support!")

## Session 1: Establishing Travel Preferences

In the first session, we'll share travel preferences with the agent. The agent will store this information in its persistent memory.

In [ ]:
prompt = """Hi! I'm Alice and I'm planning my next trip. Let me share my travel preferences:

- **Food**: I'm vegetarian and love Italian cuisine, especially pasta and risotto
- **Activities**: I enjoy art museums, historical sites, and walking tours
- **Budget**: Around $3000 for a week-long trip
- **Accommodation**: I prefer boutique hotels in central locations
- **Pace**: I like a relaxed pace with time to explore each place

Please remember these preferences for our future conversations!
"""

print_conversation("😊 User (Alice)", prompt, SESSION_1_ID)

response = invoke_agent(prompt, SESSION_1_ID, USER_ID)

print_conversation("🤖 Agent", response, SESSION_1_ID)

## Session 2: Testing Cross-Session Memory

Now we'll use a **different session ID** but the **same user ID**. The agent should remember Alice's preferences from Session 1.

In [ ]:
prompt = """Hi! I'm back. Can you remind me what you know about my travel preferences? 
And based on that, suggest a 3-day itinerary for me."""

print("\n" + "*"*80)
print("🔄 NEW SESSION - Testing Cross-Session Memory")
print("*"*80 + "\n")

print_conversation("😊 User (Alice)", prompt, SESSION_2_ID)

response = invoke_agent(prompt, SESSION_2_ID, USER_ID)

print_conversation("🤖 Agent", response, SESSION_2_ID)

print("\n✅ If the agent remembered your preferences, cross-session memory is working!")